# Image classification and denoising

### The CIFAR-10 dataset
We will work on the [**CIFAR-10 dataset**](https://www.cs.toronto.edu/~kriz/cifar.html) collected by Alex Krizhevsky, Vinod Nair, and Geoffrey Hinton from the University of Toronto.  This dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. Each image is a 3-channel colour images of 32x32 pixels in size. There are 50000 training images and 10000 test images. 

			
### Data loading and manipulation

 **Download** both the training and test data of the CIFAR-10 dataset, e.g., by following the [pytorch CIFAR10 tutorial](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html).

**Add random noise** to all training and test data to generate noisy dataset, e.g., by `torch.randn()`, with a scaling  factor `scale`, e.g., original image `+ scale * torch.randn()`, and **normalise/standardise** the pixel values to the **original range**, e.g.,  using `np.clip()`. You may choose any `scale` value between 0.2 and 0.5. 

**Note: Before generating the random noise, MUST set the random seed for reproducibility, e.g., using `torch.manual_seed()`. This seed needs to be used for all remaining code if there is randomness, for reproducibility.**

**Extract a subset** with only two classes: **Cat** and **Dog** and name it starting with **CatDog**.        

Show 10 pairs of original and noisy images of cats and 10 pairs of original and noisy images of dogs.

In [ ]:
# The code below is various methods to implement explanations above.

# Method 1
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

torch.manual_seed(140206229)
scale = 0.2
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + scale * torch.randn(tensor.size()) * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)


def get_relevant_indices(dataset, classes, target_classes):
    indices = []
    for i in range(len(dataset)):
        # Check if the label is in the target classes
        label_index = dataset[i][1] # ex: 3
        label_class = classes[label_index] # ex: 'cat'
        if label_class in target_classes:
            indices.append(i)

    return indices

def normalize(data):
    """
    Given a tensor containing 2 possible values, normalize this to 0/1

    Args:
        labels: a 1D tensor containing two possible scalar values
    Returns:
        A tensor normalize to 0/1 value
    """
    max_val = torch.max(data)
    min_val = torch.min(data)
    norm_data = (data - min_val)/(max_val - min_val)

    return norm_data

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
    

classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')

transform1 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform2 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    AddGaussianNoise(0., 1.)
])

# Download the data set
original_trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=transform1)

original_testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                           download=True, transform=transform1)

# Put random noise, normalise will be done later using fuction above
noised_trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=transform2)

noised_testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                           download=True, transform=transform2)


# Get the list of indices to sample from
cat_dog_train_indices = get_relevant_indices(original_trainset,classes,["cat", "dog"])
cat_dog_test_indices = get_relevant_indices(original_testset, classes, ["cat", "dog"])

cat_train_indices = get_relevant_indices(original_trainset,classes,["cat"])
cat_test_indices = get_relevant_indices(original_testset,classes,["cat"])

dog_train_indices = get_relevant_indices(original_trainset,classes,["dog"])
dog_test_indices = get_relevant_indices(original_testset,classes,["dog"])



cat_dog_train_sampler = SubsetRandomSampler(cat_dog_train_indices)
dog_train_sampler = SubsetRandomSampler(dog_train_indices)
cat_train_sampler = SubsetRandomSampler(cat_train_indices)

cat_dog_test_sampler = SubsetRandomSampler(cat_dog_test_indices)
dog_test_sampler = SubsetRandomSampler(dog_test_indices)
cat_test_sampler = SubsetRandomSampler(cat_test_indices)





In [ ]:
# Show 10 pairs of original/noised cats and dogs

original_dog_train_loader = torch.utils.data.DataLoader(original_trainset, batch_size=10,
                                          num_workers=1, sampler=dog_train_sampler)
noised_dog_train_loader = torch.utils.data.DataLoader(noised_trainset, batch_size=10,
                                          num_workers=0, sampler=dog_train_sampler)

original_cat_train_loader = torch.utils.data.DataLoader(original_trainset, batch_size=10,
                                          num_workers=1, sampler=cat_train_sampler)
noised_cat_train_loader = torch.utils.data.DataLoader(noised_trainset, batch_size=10,
                                          num_workers=0, sampler=cat_train_sampler)


    
# Show images of dogs
dataiter_dog1 = iter(original_dog_train_loader)
images_dog1, labels_dog2 = dataiter_dog1.next()
dataiter_dog2 = iter(noised_dog_train_loader)
images_dog2, labels_dog2 = dataiter_dog2.next()

imshow(torchvision.utils.make_grid(images_dog1))
imshow(torchvision.utils.make_grid(images_dog2))


# Show images of cats
dataiter_cat1 = iter(original_cat_train_loader)
images_cat1, labels_cat2 = dataiter_cat1.next()
dataiter_cat2 = iter(noised_cat_train_loader)
images_cat2, labels_cat2 = dataiter_cat2.next()

imshow(torchvision.utils.make_grid(images_cat1))
imshow(torchvision.utils.make_grid(images_cat2))

In [ ]:
# Method 2

# Download the data set and put random noise
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset, DataLoader

torch.manual_seed(140206229)

scale = 0.2

class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + scale * torch.randn(tensor.size()) * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

########################################################################
# The output of torchvision datasets are PILImage images of range [0, 1].
# We transform them to Tensors of normalized range [-1, 1].
transform1 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform2 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    AddGaussianNoise(0., 1.)
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform1)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform1)

classes = {'plane':0, 'car':1, 'bird':2, 'cat':3, 'deer':4, 'dog':5, 'frog':6, 'horse':7, 'ship':8, 'truck':9}


In [ ]:
# Extract a subset
x_train = trainset.data
x_test = testset.data
y_train = trainset.targets
y_test = testset.targets
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
    

def get_class_i(x, y, i):
    # Convert to a numpy array
    y = np.array(y)
    # Locate position of labels that equals to i
    pos_i = np.argwhere(y == i)
    # Convert the result into a 1-D list
    pos_i = list(pos_i[:,0])
    # Collect all data that match the desired label
    x_i = [x[j] for j in pos_i]
    
    return x_i

class DatasetMaker(Dataset):
    def __init__(self, datasets, transformFunc = transform2):
        self.datasets = datasets
        self.lengths  = [len(d) for d in self.datasets]
        self.transformFunc = transformFunc
        
    def __getitem__(self, i):
        class_label, index_wrt_class = self.index_of_which_bin(self.lengths, i)
        img = self.datasets[class_label][index_wrt_class]
        img = self.transformFunc(img)
        return img, class_label

    def __len__(self):
        return sum(self.lengths)
    
    def index_of_which_bin(self, bin_sizes, absolute_index, verbose=False):
        # Which class/bin does i fall into?
        accum = np.add.accumulate(bin_sizes)
        if verbose:
            print("accum =", accum)
        bin_index  = len(np.argwhere(accum <= absolute_index))
        if verbose:
            print("class_label =", bin_index)
        # Which element of the fallent class/bin does i correspond to?
        index_wrt_class = absolute_index - np.insert(accum, 0, 0)[bin_index]
        if verbose:
            print("index_wrt_class =", index_wrt_class)

        return bin_index, index_wrt_class

# CatDog dataset    

original_cat_dog_trainset = DatasetMaker([get_class_i(x_train, y_train, classes['cat']), 
                                 get_class_i(x_train, y_train, classes['dog'])],
                                 transform1)
original_cat_dog_testset  = DatasetMaker([get_class_i(x_test , y_test , classes['cat']),
                                 get_class_i(x_test , y_test , classes['dog'])],
                                 transform1)


noised_cat_dog_trainset = DatasetMaker([get_class_i(x_train, y_train, classes['cat']), 
                                 get_class_i(x_train, y_train, classes['dog'])],
                                 transform2)
noised_cat_dog_testset  = DatasetMaker([get_class_i(x_test , y_test , classes['cat']),
                                 get_class_i(x_test , y_test , classes['dog'])],
                                 transform2)


# 1Show 10 pairs of original/noised cat and dog
# Cats dataset
original_cat_trainset = DatasetMaker([get_class_i(x_train, y_train, classes['cat']), 
                                 get_class_i(x_train, y_train, classes['cat'])],
                                 transform1)
noised_cat_trainset = DatasetMaker([get_class_i(x_train, y_train, classes['cat']), 
                                 get_class_i(x_train, y_train, classes['cat'])],
                                 transform2)

original_cat_trainloader = torch.utils.data.DataLoader(original_cat_trainset, batch_size=10, 
                                               shuffle=True , num_workers=0)
noised_cat_trainloader = torch.utils.data.DataLoader(noised_cat_trainset, batch_size=10, 
                                               shuffle=True , num_workers=0)

# Dogs dataset
original_dog_trainset = DatasetMaker([get_class_i(x_train, y_train, classes['dog']), 
                                 get_class_i(x_train, y_train, classes['dog'])],
                                 transform1)
noised_dog_trainset = DatasetMaker([get_class_i(x_train, y_train, classes['dog']), 
                                 get_class_i(x_train, y_train, classes['dog'])],
                                 transform2)

original_dog_trainloader = torch.utils.data.DataLoader(original_dog_trainset, batch_size=10, 
                                               shuffle=True , num_workers=0)
noised_dog_trainloader = torch.utils.data.DataLoader(noised_dog_trainset, batch_size=10, 
                                               shuffle=True , num_workers=0)


# Show images of cats
dataiter_cat1 = iter(original_cat_trainloader)
images_cat1, labels_cat1 = dataiter_cat1.next()

dataiter_cat2 = iter(noised_cat_trainloader)
images_cat2, labels_cat2 = dataiter_cat2.next()
    
imshow(torchvision.utils.make_grid(images_cat1))
imshow(torchvision.utils.make_grid(images_cat2))

# Show images of dogs
dataiter_dog1 = iter(original_dog_trainloader)
images_dog1, labels_dog1 = dataiter_dog1.next()

dataiter_dog2 = iter(noised_dog_trainloader)
images_dog2, labels_dog2 = dataiter_dog2.next()
    
imshow(torchvision.utils.make_grid(images_dog1))
imshow(torchvision.utils.make_grid(images_dog2))


In [ ]:
# Method3 (recommended)

from keras.datasets import cifar10
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset, DataLoader
from keras.layers.noise import GaussianNoise

# Download the data set

torch.manual_seed(140206229)
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print('Traning data shape:', x_train.shape)
print('Testing data shape:', x_test.shape)


# Put random noise
scale = 0.2

x_train = (x_train-x_train.mean())/(x_train.max()-x_train.min())
x_test = (x_test-x_test.mean())/(x_test.max()-x_test.min())
x_train_noised = x_train + scale * np.random.normal(loc=0.0, scale=scale, size=x_train.shape)
x_test_noised = x_test + scale * np.random.normal(loc=0.0, scale=scale, size=x_test.shape)



# Extract a subset
# Original
x_train_cat = x_train[np.where(y_train==3)[0],:,:,:]
x_train_dog = x_train[np.where(y_train==5)[0],:,:,:]
x_train_cat_dog = np.concatenate((x_train_cat,x_train_dog), axis=0)

y_train_cat = y_train[np.where(y_train==3)[0],:]
y_train_dog = y_train[np.where(y_train==5)[0],:]
y_train_cat_dog = np.concatenate((y_train_cat,y_train_dog), axis=0)

x_test_cat = x_test[np.where(y_test==3)[0],:,:,:]
x_test_dog = x_test[np.where(y_test==5)[0],:,:,:]
x_test_cat_dog = np.concatenate((x_test_cat,x_test_dog), axis=0)

y_test_cat = y_test[np.where(y_test==3)[0],:]
y_test_dog = y_test[np.where(y_test==5)[0],:]
y_test_cat_dog = np.concatenate((y_test_cat,y_test_dog), axis=0)

# Noised

x_train_cat_noised = x_train_noised[np.where(y_train==3)[0],:,:,:]
x_train_dog_noised = x_train_noised[np.where(y_train==5)[0],:,:,:]
x_train_cat_dog_noised = np.concatenate((x_train_cat_noised,x_train_dog_noised), axis=0)

y_train_cat_noised = y_train[np.where(y_train==3)[0],:]
y_train_dog_noised = y_train[np.where(y_train==5)[0],:]
y_train_cat_dog_noised = np.concatenate((y_train_cat_noised,y_train_dog_noised), axis=0)

x_test_cat_noised = x_test_noised[np.where(y_test==3)[0],:,:,:]
x_test_dog_noised = x_test_noised[np.where(y_test==5)[0],:,:,:]
x_test_cat_dog_noised = np.concatenate((x_test_cat_noised,x_test_dog_noised), axis=0)

y_test_cat_noised = y_test[np.where(y_test==3)[0],:]
y_test_dog_noised = y_test[np.where(y_test==5)[0],:]
y_test_cat_dog_noised = np.concatenate((y_test_cat_noised,y_test_dog_noised), axis=0)

# Show 10 pairs of original/noised cat and dog

from mpl_toolkits.axes_grid1 import ImageGrid


def imshow(data_set):
    fig = plt.figure(figsize=(5., 5.))
    grid = ImageGrid(fig, 111,  # similar to subplot(111)
                 nrows_ncols=(2, 5),  # creates 2x2 grid of axes
                 axes_pad=0.1,  # pad between axes in inch.
                 )
    for ax, im in zip(grid, data_set):
        ax.imshow(im)

In [ ]:
imshow(x_train_cat)

In [ ]:
imshow(x_train_cat_noised)

In [ ]:
imshow(x_train_dog)

In [ ]:
imshow(x_train_dog_noised)

### Dimensionality reduction, binary classification, and evaluation

This uses the **CatDog** subset **with no noise added**.

#### Training

Apply PCA on the training set to reduce the dimensionality. **At least seven** different values for the reduced dimensionality.

**Eight** Naive Bayes classifiers: one on the original features (raw pixels), and seven on the seven different PCA features.

#### Testing and evaluation
The eight Naive Bayes classifiers explaned on the test set in terms of **classification accuracy** and **visualise** their performance using a bar graph.

Plot the [ROC Curves](https://en.wikipedia.org/wiki/Receiver_operating_characteristic) in true positive rates vs false positive rates for the eight Naive Bayes classifiers in **one figure** using eight different line/marker styles clearly labelled. 

Compute the [area under the ROC curve](https://en.wikipedia.org/wiki/Receiver_operating_characteristic#Area_under_the_curve) values for the eight Naive Bayes classifiers and visualise using a bar graph.

**At least three** interesting observations from the evaluation results above described.


In [ ]:
# Apply PCA

import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
import pandas as pd

# Reduce the dimension
x_train_cat_dog_reshape = x_train_cat_dog.reshape(x_train_cat_dog.shape[0],3*32*32)
x_test_cat_dog_reshape = x_test_cat_dog.reshape(x_test_cat_dog.shape[0],3*32*32)

y_train_cat_dog_f = y_train_cat_dog.flatten()
y_test_cat_dog_f = y_test_cat_dog.flatten()


"With wide range of number of components, can obtain wide range of results as well."
no_components1 = 50
no_components2 = 200
no_components3 = 450
no_components4 = 800
no_components5 = 1050
no_components6 = 1500
no_components7 = 2000


# Train eight Naive Bayes classifiers
model = GaussianNB()

# With original features
model.fit(x_train_cat_dog_reshape, y_train_cat_dog_f)

# With 7 pca features

def PCA_x_xt(no_components):
    pca = PCA(n_components=no_components, svd_solver='randomized', whiten=True)
    
    x_pca_2d = pca.fit_transform(x_train_cat_dog_reshape)
    xt_pca_2d = pca.transform(x_test_cat_dog_reshape)
    
    model.fit(x_pca_2d, y_train_cat_dog_f)
    
    X_pred = model.predict(xt_pca_2d)
    
    return X_pred

# Test and visualise

# Naive Bayes with raw pixels
X_pred1 = model.predict(x_test_cat_dog_reshape)

pd_results = pd.DataFrame([
        ['No PCA', metrics.accuracy_score(y_test_cat_dog_f, X_pred1)],
        [no_components1, metrics.accuracy_score(y_test_cat_dog_f, PCA_x_xt(no_components1))],
        [no_components2, metrics.accuracy_score(y_test_cat_dog_f, PCA_x_xt(no_components2))],
        [no_components3, metrics.accuracy_score(y_test_cat_dog_f, PCA_x_xt(no_components3))],
        [no_components4, metrics.accuracy_score(y_test_cat_dog_f, PCA_x_xt(no_components4))],
        [no_components5, metrics.accuracy_score(y_test_cat_dog_f, PCA_x_xt(no_components5))],
        [no_components6, metrics.accuracy_score(y_test_cat_dog_f, PCA_x_xt(no_components6))],
        [no_components7, metrics.accuracy_score(y_test_cat_dog_f, PCA_x_xt(no_components7))]
    ], columns=['Number of PCA Components', 'Accuracy'])
pd_results.plot(kind='barh', x='Number of PCA Components', y='Accuracy')


In [ ]:
# Plot ROC curve (label = dog)

def roc_get_fp_tp(no_components):
    pred = PCA_x_xt(no_components)
    fpr, tpr, _ = metrics.roc_curve(y_test_cat_dog_f, pred, pos_label=5)
    
    return fpr, tpr

fpr2, tpr2 = roc_get_fp_tp(no_components1)
fpr3, tpr3 = roc_get_fp_tp(no_components2)
fpr4, tpr4 = roc_get_fp_tp(no_components3)
fpr5, tpr5 = roc_get_fp_tp(no_components4)
fpr6, tpr6 = roc_get_fp_tp(no_components5)
fpr7, tpr7 = roc_get_fp_tp(no_components6)
fpr8, tpr8 = roc_get_fp_tp(no_components7)


# For NB without PCA
fpr, tpr, _ = metrics.roc_curve(y_test_cat_dog_f, X_pred1, pos_label=5)

# Plot the results
fig, ax = plt.subplots(1, figsize=(12, 6))
plt.plot(fpr, tpr, color='cyan', label='No PCA')
plt.plot(fpr2, tpr2, color='blue', label='ROC curve pca:50')
plt.plot(fpr3, tpr3, color='red', label='ROC curve pca:200')
plt.plot(fpr4, tpr4, color='orange', label='ROC curve pca:450')
plt.plot(fpr5, tpr5, color='black', label='ROC curve pca:800')
plt.plot(fpr6, tpr6, color='yellow', label='ROC curve pca:1050')
plt.plot(fpr7, tpr7, color='green', label='ROC curve pca:1500')
plt.plot(fpr8, tpr8, color='magenta', label='ROC curve pca:2000')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate (1 - specificity)')
plt.ylabel('True Positive Rate (sensitivity)')
plt.title('ROC Curve for CIFAR10')
plt.legend(loc="lower right")

In [ ]:
# Plot AUC

pd_results = pd.DataFrame([
        ['No PCA', metrics.auc(fpr, tpr)],
        [no_components1, metrics.auc(fpr2, tpr2)],
        [no_components2, metrics.auc(fpr3, tpr3)],
        [no_components3, metrics.auc(fpr4, tpr4)],
        [no_components4, metrics.auc(fpr5, tpr5)],
        [no_components5, metrics.auc(fpr6, tpr6)],
        [no_components6, metrics.auc(fpr7, tpr7)],
        [no_components7, metrics.auc(fpr8, tpr8)]
    ], columns=['Number of PCA Components', 'AUC'])
pd_results.plot(kind='barh', x='Number of PCA Components', y='AUC')

# Observations

# 1. The accuracy without applying PCA is still fairly high. This may tell applying PCA is not always 
# helpful. By itself, it can give good result and works well.

# 2. Some of PCA applied showed lower accuracies than the one that's not. This may tell applying not all
# ks are helpful, but only specific number. To find those good ks, it needs several attempts with different/wide range of k values

# 3. Currently, the highest accuracy is from when k = 800 and the lowest is when k = 1500. Having too much ks is not always correct answer.
#  The AUC is shows same trend as the accuracy score



### Noisy data and multiclass classification

#### Noisy **CatDog** subset.

Repeat process above on the noisy version of CatDog subset.

#### Multiclass classification using the original CIFAR-10 dataset (all 10 classes)

Apply PCA on the training set to reduce the dimensionality.

Train nine classifers: **four Naive Bayes** classifiers(one on the original features, and three on the three different PCA features in 3b); **four Logistic Regression** classifiers (one on the original features, and three on the three different PCA features in 3b); and one **Convoluational Neural Network** as defined in the [pytorch CIFAR10 tutorial](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html).

Evalaute the nine classifiers on the test set. Summarised the **classification accuracy**, **total training time**, and **total test time** using three bar graphs.

The confusion matrix for these nine classifiers shown

Described **at least three** interesting observations from the evaluation results above.

In [ ]:
# Repeat above for noisy dataset

# Reduce dimensions
x_train_cat_dog_noised_reshape = x_train_cat_dog_noised.reshape(x_train_cat_dog_noised.shape[0],3*32*32)
x_test_cat_dog_noised_reshape = x_test_cat_dog_noised.reshape(x_test_cat_dog_noised.shape[0],3*32*32)

y_train_cat_dog_noised_f = y_train_cat_dog_noised.flatten()
y_test_cat_do_noised_f = y_test_cat_dog_noised.flatten()

no_components1 = 50
no_components2 = 200
no_components3 = 450
no_components4 = 800
no_components5 = 1050
no_components6 = 1500
no_components7 = 2000


# With original features
model.fit(x_train_cat_dog_reshape, y_train_cat_dog_f)


# Test and visualise

X_pred1 = model.predict(x_test_cat_dog_reshape)

pd_results = pd.DataFrame([
        ['No PCA', metrics.accuracy_score(y_test_cat_dog_f, X_pred1)],
        [no_components1, metrics.accuracy_score(y_test_cat_dog_f, PCA_x_xt(no_components1))],
        [no_components2, metrics.accuracy_score(y_test_cat_dog_f, PCA_x_xt(no_components2))],
        [no_components3, metrics.accuracy_score(y_test_cat_dog_f, PCA_x_xt(no_components3))],
        [no_components4, metrics.accuracy_score(y_test_cat_dog_f, PCA_x_xt(no_components4))],
        [no_components5, metrics.accuracy_score(y_test_cat_dog_f, PCA_x_xt(no_components5))],
        [no_components6, metrics.accuracy_score(y_test_cat_dog_f, PCA_x_xt(no_components6))],
        [no_components7, metrics.accuracy_score(y_test_cat_dog_f, PCA_x_xt(no_components7))]
    ], columns=['Number of PCA Components', 'Accuracy'])
pd_results.plot(kind='barh', x='Number of PCA Components', y='Accuracy')




In [ ]:
from sklearn.linear_model import LogisticRegression
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import time

# Apply pca to original data

x_train_reshape = x_train.reshape(x_train.shape[0],3*32*32)
x_test_reshape = x_test.reshape(x_test.shape[0],3*32*32)

y_train_f = y_train.flatten()
y_test_f = y_test.flatten()

# From the previous results, these 3 number of components showed the highest accuracies, therefore chosen
no_components1 = 50
no_components2 = 800
no_components3 = 2000


# 9 classifiers
# 1 Naive Bayes Original
NB_model = GaussianNB()

# With original features
NB_model.fit(x_train_reshape, y_train_f)

# 3 Naive Bayes PCA
def PCA_NB_x_xt_2(no_components):
    start_time = time.time()
    pca = PCA(n_components=no_components, svd_solver='randomized', whiten=True)
    
    x_pca_2d = pca.fit_transform(x_train_reshape)
    xt_pca_2d = pca.transform(x_test_reshape)
    
    NB_model.fit(x_pca_2d, y_train_f)
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    start_time2 = time.time()
    X_pred_2 = NB_model.predict(xt_pca_2d)
    end_time2 = time.time()
    elapsed_time2 = end_time2 - start_time2
    
    return X_pred_2,elapsed_time,elapsed_time2

# 1 Logistic Regression Original
LR_model = LogisticRegression(max_iter=3, verbose=True)
LR_model.fit(x_train_reshape,y_train_f)

# 3 Logistic Regression PCA
def PCA_LR_x_xt_2(no_components):
    start_time = time.time()
    pca = PCA(n_components=no_components, svd_solver='randomized', whiten=True)
    
    x_pca_2d = pca.fit_transform(x_train_reshape)
    xt_pca_2d = pca.transform(x_test_reshape)
    
    LR_model.fit(x_pca_2d, y_train_f)
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    start_time2 = time.time()
    X_pred_3 = LR_model.predict(xt_pca_2d)
    end_time2 = time.time()
    elapsed_time2 = end_time2 - start_time2
    return X_pred_3,elapsed_time,elapsed_time2

# CNN
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) #3: #input channels; 6: #output channels; 5: kernel size
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

max_epochs=2
start_time_nn = time.time()
for epoch in range(max_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')
end_time_nn = time.time()
elapsed_time_nn_tr = end_time_nn - start_time_nn

PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

In [ ]:
# Evalaute the nine classifiers on the test set
result1_pred, result1_tr_t, result1_t_t = PCA_NB_x_xt_2(no_components1)
result2_pred, result2_tr_t, result2_t_t = PCA_NB_x_xt_2(no_components2)
result3_pred, result3_tr_t, result3_t_t = PCA_NB_x_xt_2(no_components3)
result4_pred, result4_tr_t, result4_t_t = PCA_LR_x_xt_2(no_components1)
result5_pred, result5_tr_t, result5_t_t = PCA_LR_x_xt_2(no_components2)
result6_pred, result6_tr_t, result6_t_t = PCA_LR_x_xt_2(no_components3)


start_time_NB_tr = time.time()
NB_model.fit(x_train_reshape, y_train_f)
end_time_NB_tr = time.time()
elasped_time_NB_tr = end_time_NB_tr - start_time_NB_tr


start_time_NB_t = time.time()
result0_pred = NB_model.predict(x_test_reshape)
end_time_NB_t = time.time()
elasped_time_NB_t = end_time_NB_t - start_time_NB_t



start_time_LR_tr = time.time()
LR_model.fit(x_train_reshape,y_train_f)
end_time_LR_tr = time.time()
elasped_time_LR_tr = end_time_LR_tr - start_time_LR_tr


start_time_LR_t = time.time()
result7_pred = LR_model.predict(x_test_reshape)
end_time_LR_t = time.time()
elasped_time_LR_t = end_time_LR_t - start_time_LR_t


start_time_nn_t = time.time()
net = Net()
net.load_state_dict(torch.load(PATH))

correct = 0
total = 0
with torch.no_grad():  #testing phase, no need to compute the gradients to save time
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

end_time_nn_t = time.time()
elapsed_time_nn_t = end_time_nn_t - start_time_nn_t

accuracies = [metrics.accuracy_score(y_test_f, result0_pred),
              metrics.accuracy_score(y_test_f, result1_pred),
              metrics.accuracy_score(y_test_f, result2_pred),
              metrics.accuracy_score(y_test_f, result3_pred),
              metrics.accuracy_score(y_test_f, result7_pred),
              metrics.accuracy_score(y_test_f, result4_pred),
              metrics.accuracy_score(y_test_f, result5_pred),
              metrics.accuracy_score(y_test_f, result6_pred),
              correct / total]
total_train_time = [elasped_time_NB_tr,result1_tr_t,result2_tr_t,result3_tr_t,
                    elasped_time_NLR_tr,result4_tr_t,result5_tr_t,result6_tr_t,
                    elapsed_time_nn_tr]
total_test_time = [elasped_time_NB_t,result1_t_t,result2_t_t,result3_t_t,
                   elasped_time_LR_t,result4_t_t,result5_t_t,result6_t_t,
                   elapsed_time_nn_t]

index = ['NB_Original', 'NB_PCA1', 'NB_PCA2','NB_PCA3',
         'LR_Original', 'LR_PCA1', 'LR_PCA2','LR_PCA3',
         'CNN']


In [ ]:
total_train_time = [elasped_time_NB_tr,result1_tr_t,result2_tr_t,result3_tr_t,
                    elasped_time_LR_tr,result4_tr_t,result5_tr_t,result6_tr_t,
                    elapsed_time_nn_tr]
total_test_time = [elasped_time_NB_t,result1_t_t,result2_t_t,result3_t_t,
                   elasped_time_LR_t,result4_t_t,result5_t_t,result6_t_t,
                   elapsed_time_nn_t]

index = ['NB_Original', 'NB_PCA1', 'NB_PCA2','NB_PCA3',
         'LR_Original', 'LR_PCA1', 'LR_PCA2','LR_PCA3',
         'CNN']

In [ ]:
df = pd.DataFrame({'accuracies': accuracies}, index=index)
df.plot.bar(rot=0)

In [ ]:
df = pd.DataFrame({'total_train_time': total_train_time}, index=index)
df.plot.bar(rot=0)

In [ ]:
df = pd.DataFrame({'total_test_time':total_test_time}, index=index)
df.plot.bar(rot=0)

In [ ]:
# Confusion matrix

from sklearn.metrics import confusion_matrix
import seaborn as sns; sns.set() # for statistical data visualization

# for NB Original
mat1 = confusion_matrix(y_test_f, NB_model.predict(x_test_reshape))
# for NB PCA
mat2 = confusion_matrix(y_test_f, PCA_NB_x_xt_2(no_components1)[0])
mat3 = confusion_matrix(y_test_f, PCA_NB_x_xt_2(no_components2)[0])
mat4 = confusion_matrix(y_test_f, PCA_NB_x_xt_2(no_components3)[0])
# for LR Original
mat5 = confusion_matrix(y_test_f, LR_model.predict(x_test_reshape))
# for LR PCA
mat6 = confusion_matrix(y_test_f, PCA_LR_x_xt_2(no_components1)[0])
mat7 = confusion_matrix(y_test_f, PCA_LR_x_xt_2(no_components2)[0])
mat8 = confusion_matrix(y_test_f, PCA_LR_x_xt_2(no_components3)[0])


In [ ]:
sns.heatmap(mat1.T, square=True, annot=True, fmt='n', annot_kws={"size": 12})
plt.xlabel('true label')
plt.ylabel('predicted label');

In [ ]:
sns.heatmap(mat2.T, square=True, annot=True, fmt='n', annot_kws={"size": 12})
plt.xlabel('true label')
plt.ylabel('predicted label');

In [ ]:
sns.heatmap(mat3.T, square=True, annot=True, fmt='n', annot_kws={"size": 12})
plt.xlabel('true label')
plt.ylabel('predicted label');

In [ ]:
sns.heatmap(mat4.T, square=True, annot=True, fmt='n', annot_kws={"size": 12})
plt.xlabel('true label')
plt.ylabel('predicted label');

In [ ]:
sns.heatmap(mat5.T, square=True, annot=True, fmt='n', annot_kws={"size": 12})
plt.xlabel('true label')
plt.ylabel('predicted label');

In [ ]:
sns.heatmap(mat6.T, square=True, annot=True, fmt='n', annot_kws={"size": 12})
plt.xlabel('true label')
plt.ylabel('predicted label');

In [ ]:
sns.heatmap(mat7.T, square=True, annot=True, fmt='n', annot_kws={"size": 12})
plt.xlabel('true label')
plt.ylabel('predicted label');

In [ ]:
sns.heatmap(mat8.T, square=True, annot=True, fmt='n', annot_kws={"size": 12})
plt.xlabel('true label')
plt.ylabel('predicted label');

In [ ]:
# Evaluation

# 1. For both with/without noise on train data, the result shows quite similarly. 
# Both results showed when k is 50, 800 and 2000 showed higher accuracy than without PCA.
# 2. In Naive Bayes, bigger the k for PCA, longer training time and testing time. Number of k changes the results
# of how similar each classes are, so the accuracy decreased.
# 3. In Linear Regressio, it showed similar accuracies, however the difference came out whether PCA is applied or not.
# Without PCA took even longer than with PCA in total training time. Nut once trained, barely no time consumed for testing
# 4. CNN showed the best accuracy out of other classifers. However, it took quite a long training time and the longest testing time
# out of all the classifiers



### Denoising Autoencoder

This uses both the original and noisy CIFAR-10 datasets (all 10 classes).

Read about denoising autoencoder at [Wikepedia](https://en.wikipedia.org/wiki/Autoencoder#Denoising_autoencoder_(DAE)) and this [short introduction](https://towardsdatascience.com/denoising-autoencoders-explained-dbb82467fc2)

Autoencoder architecture used so that it takes colour images as input (i.e., 3 input channels). 

**Training**: feed the **noisy training images** as input to the autoencoder; use a loss function that computes the reconstruction error between the **output of the autoencoder** and the respective **original images**.

**Testing**: evaluate the autoencoder trained from above on the test datasets (feed noisy images in and compute reconstruction errors on original clean images. Find the **worstly denoised** 30 images (those with the largest reconstruction errors) in the test set and show them in pairs with the original images (60 images to show in total).

At least two hyperparameters chosen to vary. Study **at least three different choices** for each hyperparameter. When varying one hyperparameter, all the other hyperparameters can be fixed. Visualise the performance sensitivity with respect to these hyperparameters.

In [ ]:
# Autoencoder

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision import datasets, transforms

class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + scale * torch.randn(tensor.size()) * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

def normalize(data):
    """
    Given a tensor containing 2 possible values, normalize this to 0/1

    Args:
        labels: a 1D tensor containing two possible scalar values
    Returns:
        A tensor normalize to 0/1 value
    """
    max_val = torch.max(data)
    min_val = torch.min(data)
    norm_data = (data - min_val)/(max_val - min_val)

    return norm_data

classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')

transform1 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform2 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    AddGaussianNoise(0., 1.)
])

# Download the data set
original_trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=transform1)

original_testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                           download=True, transform=transform1)

# 1b. Put random noise, normalise will be done later using fuction above
noised_trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=transform2)

noised_testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                           download=True, transform=transform2)




class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            # 1 input image channel, 16 output channel, 3x3 square convolution
            nn.Conv2d(3, 16, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 7)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 7),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, 3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()  #to range [0, 1]
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

myAE=Autoencoder()

# Training noisy dataset

#Hyperparameters for training
batch_size=10
learning_rate=0.05
max_epochs = 2

#Choose mean square error loss
criterion = nn.MSELoss() 
#Choose the Adam optimiser
optimizer = torch.optim.Adam(myAE.parameters(), lr=learning_rate, weight_decay=1e-5)
#Specify how the data will be loaded in batches (with random shffling)
noised_train_loader = torch.utils.data.DataLoader(noised_trainset, batch_size=batch_size, shuffle=True)
original_train_loader = torch.utils.data.DataLoader(original_trainset, batch_size=batch_size, shuffle=True)

noised_testloader = torch.utils.data.DataLoader(noised_testset, batch_size=batch_size,shuffle=False, num_workers=2)
original_testloader = torch.utils.data.DataLoader(original_testset, batch_size=batch_size,shuffle=False, num_workers=2)

#Storage
outputs = []


#Start training
for epoch in range(max_epochs):
    for data,data2 in zip(noised_train_loader,original_train_loader):
        img, label = data
        img2, label2 = data2
        img = normalize(img)
        img2= normalize(img2)
        optimizer.zero_grad()
        recon = myAE(img)
        loss = criterion(recon, img2)
        loss.backward()
        optimizer.step()            
    if (epoch % 3) == 0:
        print('Epoch:{}, Loss:{:.4f}'.format(epoch+1, float(loss)))
    outputs.append((loss, img, recon),)
    

    


In [ ]:
# Testing

PATH = './cifar_myAE.pth'
torch.save(myAE.state_dict(), PATH)

myAE.load_state_dict(torch.load(PATH))

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))


for epoch in range(max_epochs):
    for data,data2 in zip(noised_testloader,original_testloader):
        img, label = data
        img2, label2 = data2
        img = normalize(img)
        img2= normalize(img2)
        optimizer.zero_grad()
        recon = myAE(img)
        loss = criterion(recon, img2)
        loss.backward()
        optimizer.step()            
    if (epoch % 3) == 0:
        print('Epoch:{}, Loss:{:.4f}'.format(epoch+1, float(loss)))
    outputs.append((loss, img, recon),)

numImgs=30;
for k in range(0, max_epochs, 9):
    plt.figure(figsize=(numImgs, 2))
    loss = outputs[k][0].detach().numpy()
    imgs = outputs[k][1].detach().numpy() 
    recon = outputs[k][2].detach().numpy()
    for i, item in enumerate(imgs):
        if i >= numImgs: break
        plt.subplot(2, numImgs, i+1)
        plt.imshow(item[0])
        
    for i, item in enumerate(recon):
        if i >= numImgs: break
        plt.subplot(2, numImgs, numImgs+i+1)
        plt.imshow(item[0])

In [ ]:
# Different Hyperparameters

# 1
#Hyperparameters for training
batch_size=10
learning_rate=0.005
# max_epochs = 10

optimizer = torch.optim.Adam(myAE.parameters(), lr=learning_rate, weight_decay=1e-5)

#Storage
outputs = []

#Start training
for epoch in range(max_epochs):
    for data,data2 in zip(noised_train_loader,original_train_loader):
        img, label = data
        img2, label2 = data2
        img = normalize(img)
        img2= normalize(img2)
        optimizer.zero_grad()
        recon = myAE(img)
        loss = criterion(recon, img2)
        loss.backward()
        optimizer.step()            
    if (epoch % 3) == 0:
        print('Epoch:{}, Loss:{:.4f}'.format(epoch+1, float(loss)))
    outputs.append((epoch, img, recon),)
    
PATH = './cifar_myAE.pth'
torch.save(myAE.state_dict(), PATH)

myAE.load_state_dict(torch.load(PATH))

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

In [ ]:
# 2
#Hyperparameters for training
batch_size=30
learning_rate=0.005
max_epochs = 15

optimizer = torch.optim.Adam(myAE.parameters(), lr=learning_rate, weight_decay=1e-5)

#Storage
outputs = []

#Start training
for epoch in range(max_epochs):
    for data,data2 in zip(noised_train_loader,original_train_loader):
        img, label = data
        img2, label2 = data2
        img = normalize(img)
        img2= normalize(img2)
        optimizer.zero_grad()
        recon = myAE(img)
        loss = criterion(recon, img2)
        loss.backward()
        optimizer.step()            
    if (epoch % 3) == 0:
        print('Epoch:{}, Loss:{:.4f}'.format(epoch+1, float(loss)))
    outputs.append((epoch, img, recon),)
    
PATH = './cifar_myAE.pth'
torch.save(myAE.state_dict(), PATH)

myAE.load_state_dict(torch.load(PATH))

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

In [ ]:
# 3
#Hyperparameters for training
batch_size=10
learning_rate=0.00005
max_epochs = 20

optimizer = torch.optim.Adam(myAE.parameters(), lr=learning_rate, weight_decay=1e-5)

#Storage
outputs = []

#Start training
for epoch in range(max_epochs):
    for data,data2 in zip(noised_train_loader,original_train_loader):
        img, label = data
        img2, label2 = data2
        img = normalize(img)
        img2= normalize(img2)
        optimizer.zero_grad()
        recon = myAE(img)
        loss = criterion(recon, img2)
        loss.backward()
        optimizer.step()            
    if (epoch % 3) == 0:
        print('Epoch:{}, Loss:{:.4f}'.format(epoch+1, float(loss)))
    outputs.append((epoch, img, recon),)
    
PATH = './cifar_myAE.pth'
torch.save(myAE.state_dict(), PATH)

myAE.load_state_dict(torch.load(PATH))

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

In [ ]:
# Observation
# 1. Learning rate is the most important hyperparameter. If it's too small, it takes long training process and could get stuck
#  but if it's to high, it may result in learning a sub-optimal set of weights too fast or an unstable training process.
# 2. The balance between learning rate and the maximum number of epoches is important to find the best accuracy